# Tutorial 1) Parameterizing sequences with arbok

## 0. Introduction

This tutorial gives a first insight into the architecture and philosophy behind arbok.

Arbok is a top-level python control framework based on [QCoDeS](https://microsoft.github.io/Qcodes/index.html) compiling into FPGA instructions ([QM-QUA SDK](https://pypi.org/project/qm-qua/1.1.7/)) for [quantum machines hardware](https://www.quantum-machines.co/). The core idea behind arbok is to write **qubit control sequences in a device and measurement setup agnostic manner** that are configured/ scaled to larger systems by providing the respective **configurations that characterize that given system**.

**QCoDeS** is a full stack data acquisition framework that handles instrument communication, parameterization, data storage and visualization. Arbok leverages this existing infrastructure and 

<img src="images/qcodes_logo.png" width="250">

The **following tutorial** proivides:
1. An overview of the basic architecture of arbok
2. Demonstration of building a custom square pulse from scratch
3. Defining parameter sweeps
4. Using arbok QUA helpers to write sequences

The **other tutorials** in this documentation cover the topics:
- Tutorial 0) Measurement notebook example
- Tutorial 2) Readout sequences and abstract readouts
- Tutorial 3) Input streaming of parameters

## 1. The basic architecture

To get started, four types of classes are needed:

`Sample`:\
The Sample holds the quantum machines configuration that is quantifying elements, waveforms, mixers, etc. Also contains information about voltage dividers between the quantum machine outputs and the given device. Each of the following classes requires an instance of a sample to be instanciated.

`ArbokDriver`:\
This instance can be understood as the actual qcodes instrument and manages the hardware connection, and all modular (Sub)Sequences.

`SubSequence`:\
The SubSequence has and modular snippet of qua code which is written in a device agnostic way and can be re used on any system. The SubSequence is then parameterized by a configuration in the form a python dictionairy that make the executed code device specific.

`Measurement`:\
The Measurement does not contain qua code itself but holds one or more SubSequences that are compiled together. A Measurement is meant to asseble its sub-modules to a full experiment. 


How the mentioned classes relate to each other is sketched in the schematic below:

<img src="images/arbok_scheme.svg" width="400">

## 2. Parameterizing a simple square pulse

Firstly we import `ArbokDriver`, `Sample` and `Sequence` classes that were discussed before. As a `SubSequence`, we import one of the given examples. We are starting with a simple square pulse sequence.

We also import a dummy configuration which has been taken from the quantum machines github repository.

In [ ]:
%load_ext rich

In [ ]:
import logging
logging.basicConfig(
    filename = 'logs/tute1.txt',
    filemode = 'w',
    encoding = 'utf-8',
    level = logging.DEBUG
)


In [ ]:
from arbok_driver import ArbokDriver, Sample, Measurement
from example_sequences.square_pulse import SquarePulse
from example_configs.configuration import qm_config

In [15]:
display(HTML('<img src="./figures/schematic_nested_square.svg" width="500"/>'))

### 2.1 Configuring the `Sample`

In the first step, a  `Sample` object will be configured and is used in every following
sequence.
The sample holds the configuration of the quantum machine that you probably already have from your experiments.

Besides this configuration a further **'divider_config'** is required.
This dictionary represents voltage divider that are in between the quantum machine and your sample. All voltage values in arbok are meant to be what is applied to the sample, not the output of the machine. This is implemented by the scale attribute of qcodes parameters.

A simple example is given below. A dict entry is required for every element in the quantum machines config that has an output port configured. For every element we multiply the factor `opx_scale` which compensates the output range of the OPX (-0.5V ->  0.5V)

In [ ]:
opx_scale = 2
divider_config = {
    'gate_1': {
        'division': 1*opx_scale,
    },
    'gate_2': {
        'division': 1*opx_scale,
    },
    'readout_element': {
        'division': 1*opx_scale
    }
}

Both of those configs are now used to instantiate the given sample.

In [ ]:
dummy_sample = Sample('dummy_sample', qm_config, divider_config)

### 2.2 Building the `Arbok_driver` and a `Measurement`

The sample we created previously is the only requirement to build a basic arbok_driver.

In [ ]:
qm_driver = ArbokDriver('qm_driver', dummy_sample)

`Measurement`s can now be registered into this driver. Measurements are meant to act as a container for all sub-sequences that are required to run a single type of measurement.
Currently single sequences per driver are supported.

In [ ]:
dummy_sequence = Measurement(qm_driver, 'dummy_squence', dummy_sample)

## 2.1 Configuring a simple square pulse sequence

So far, predefined classes were created and instantiated. The following `SquarePulse` class will demonstrate inheritance, let us have a look at the source code.

The `SquarePulse` inherits from `SubSequence` and the only thing added/overwritten is the `qua_sequence` method.
The qua commands to execute are written within that method and the given arguments are filled with other attribute calls (like self.amplitude()).
Those attributes are in fact qcodes parameters and can be tracked and varied throughout an experiment.

In [ ]:
from example_sequences.square_pulse import SquarePulse
SquarePulse??

The `SquarePulse` `SubSequence` requires parameters (e.g. defined in a `sequence_config` file) that adds a `qcodes.parameter` for each entry like the one given here (note that we import `Voltage` and `Time` to specify our parameter units and other variables automatically):

In [ ]:
from arbok_driver.parameter_types import Voltage, Time, String

square_conf = {
    'amplitude': {
        'value': 0.5,
        'type': Voltage,
    },
    't_square_pulse': {
        'value': 100,
        'type': Time
    },
    'element': {
        'value': 'gate_1',
        'unit': 'gate label',
        'type':  String
    },
    'ramp_time': {
        'value': 20,
        'type': Time
    },
}

`square_conf` fully configures parameters with an initial value and
unit. Optionally you can add the **variable type within qua, an axis label for data saving and validators** like this:

In [ ]:
from qcodes.validators import Ints
_ = {
    't_square_pulse': {
        'value': 100,
        'type': Time,
        'label': 'Square pulse width'
    },
}

After creating the square pulse `SubSequence` with the respective config we can take a look at its snapshot.

Again, as all (sub-) sequences so far it requires a sample object for instantiation.
This makes sure that all added sub-sequences are configured for the same device.
Finally we add the `SquarePulse` of type `SubSequence` to our `Sequence` which is registered to the `ArbokDriver`.

In [ ]:
square_pulse = SquarePulse(dummy_sequence, 'square_pulse', dummy_sample, square_conf)

In [ ]:
qm_driver.dummy_squence.square_pulse.print_readable_snapshot()

Another way to see all available parameters on a (sub-) sequence is by checking the parameters attribute:

square_pulse.parameters

Parameters can be easily modified and read out. Let us half the duration of the square pulse.

In [ ]:
square_pulse.t_square_pulse.set(50)

In [ ]:
square_pulse.t_square_pulse.get()

### 2.3 Compiling the qua program

All (sub-) sequences are prepared and the QUA program can be compiled. The method `get_qua_program` of the `ArbokDriver` recursively goes through the sequences and subsequences and returns the qua source code. The output can now be printed to a file with the method `print_qua_program_to_file`. Let us have a look at the result:

In [ ]:
qua_program = qm_driver.get_qua_program()

qm_driver.print_qua_program_to_file(
    'qua_programs/tut1_square_pulses.py', qua_program)
from qua_programs import tut1_square_pulses
tut1_square_pulses??

The implicit parameters of the `SquarePulse` class became explicit upon QUA compilation with the values we have set.

Two lines defining an `infinite_loop` and `pause` of the given QUA program were not written in the square pulse class and are created by the driver in every case. The given logic is required to perform multiple shots of a sequence for example with different parameters or just to average a result. The `pause` statement keeps the client PC in sync with the execution of the quantum machine for live data plotting, input streaming, etc.

## 3. Parameter sweeps

So far we have learnt how to create a qcodes Instrument which parameterises a QUA sequence. Often experiments are characterising a physical system by sweeping parameters. In the next step we want to get familiar with defining parameter sweeps. Parameters can be swept concurrently along the same axis and the sweeps can be nested arbitrarily deep.

In [ ]:
import numpy as np

First we look at the most simple example of a 1D sweep (a sweep of only one parameter). Remember that the sequence acts as the container for all subseqeunces and is meant to represent one experiment. Therefore we define the sweep on it.

In [ ]:
dummy_sequence.set_sweeps(
    {
        square_pulse.amplitude: np.linspace(0.1, 1, 5)
    }
)

We print the compiled program to a file to inspect. One finds that the `SquarePulse` instructions are indented by an additional for loop which represents the defined sweep. In comparison to the example above, the parameter we sweep is not defined explicitly anymore but given by a QUA variable everywhere it is called.

Arbok tries to parameterize input arrays into **start, step and stop** to save FPGA memory. A user warning is raised every time this is done. The threshold for parameterisation for this is currently that the variance of the input array item steps is 10 times smaller than the step size itself. Arbok will always raise a warning when input arrays are parameterised.

In [ ]:
qua_program = qm_driver.get_qua_program()

qm_driver.print_qua_program_to_file('qua_programs/tut1_parameter_sweeps.py', qua_program)
from qua_programs import tut1_parameter_sweeps
tut1_parameter_sweeps??

Now let us see how a two dimensional sweep would work where we sweep two parameters along a certain axis. When sweeping two params together it is important to pass input arrays that have the same length.

In [ ]:
dummy_sequence.set_sweeps(
    {
        square_pulse.amplitude: np.linspace(0.1, 1, 5)
    },
    {
        square_pulse.t_square_pulse: np.arange(10, 50, 10, dtype = int),
        square_pulse.ramp_time: np.arange(10, 50, 10, dtype = int)
    }
)

We see that there is yet another for loop nested into the other. For multi param sweeps, arrays are always defined explicitly.

In [ ]:
qua_program = qm_driver.get_qua_program()

qm_driver.print_qua_program_to_file(
    'qua_programs/tut1_multi_param_sweeps.py', qua_program)
from qua_programs import tut1_multi_param_sweeps
tut1_multi_param_sweeps??

In [ ]:
square_pulse.print_readable_snapshot()

## 4. QUA helpers for system scale up

### 4.1 Creating parameters for multiple elements at the same time

If multiple output gates have to be configured at the same time, for example when one wants to move in a high dimensional voltage space, parameters can be defined with a gate prefix as shown below. Looking at 'vHome' and 'vSquare' you will see that no 'value' key is given here but 'elements'. For this given configuration four different parameters will be defined for both 'vHome' and 'vSquare' as seen in the snapshot below.

In [ ]:
from example_sequences.square_pulse2 import SquarePulse2
from arbok_driver.parameter_types import List

In [ ]:
square_conf2 = {
    'sticky_elements': {
        'value': ['gate_1', 'gate_2', 'gate_3', 'gate_4'],
        'unit': 'gate label',
        'type': List
    },
    'vHome': {
        'type': Voltage,
        "label": 'Default voltage point during the sequence',
        'elements': {
            'gate_1': 0,
            'gate_2': 0,
            'gate_3': 0,
            'gate_4': 0,
        }
    },
    'vSquare': {
        'type': Voltage,
        "label": 'Voltage amplitude of square pulse',
        'elements': {
            'gate_1': 0.1,
            'gate_2': -0.05,
            'gate_3': 0.08,
            'gate_4': 0.25,
        }
    },
    't_square_pulse': {
        'value': 100,
        'type': Time
    },
    'ramp_time': {
        'value': 20,
        'type': Time
    },
}

In [ ]:
qm_driver2 = ArbokDriver('qm_driver2', dummy_sample, square_conf2)
dummy_sequence2 = Sequence(qm_driver2, 'dummy_sequence2', dummy_sample)
square_pulse2 = SquarePulse2(dummy_sequence2, 'square_pulse2', dummy_sample, square_conf2)

After instantiating `SquarePulse2` with the given configuration we see that for example 'vSquare' created one paramer for each given gate. Resulting parameters are 'vSquare_gate_1', 'vSquare_gate_2', 'vSquare_gate_3', 'vSquare_gate_4'.

In [ ]:
square_pulse2.print_readable_snapshot()

### 4.2 Playing pulses on multiple gates at the same time

The square pulse we imported now is slightly different to the one we have seen before. Play statements are not called explicitly but with the qua_helper arbok_go that takes the gate prefix and a list of gates. Play commands are then executed for all of those combinations that are available. Explicit align statements with the given gates can be executed optionally as well. See the resulting program for reference below.

In [ ]:
square_pulse2??

In [ ]:
qua_program = qm_driver2.get_qua_program()

qm_driver.print_qua_program_to_file('qua_programs/tut1_scale_up.py', qua_program)
from qua_programs import tut1_scale_up
tut1_scale_up??